In [1]:
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qmi

# Indices correct?

compare indices for manually created small atomic kernel with full kernel

In [2]:
data, molecule_size = qmi.load_alchemy_data(qmi.wrapper_alch_data())

In [3]:
# choose random molecules
global_tr, global_test = qmi.get_indices(len(molecule_size), 3)
global_tr = np.array([ 14, 519, 549])
tr_ind = qmi.get_local_idx(global_tr, molecule_size)
test_ind = qmi.get_local_idx(global_test, molecule_size)

In [4]:
# calculate representation and atomic kernels
reps = []
for i in global_tr:
    rep_fchl = qml.fchl.generate_representation(data[i][:, 1:4], data[i][:,0], max_size=molecule_size[i])
    reps.append(rep_fchl)

In [11]:
sigma = 432.8761281083057
kernel00 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[0], reps[0], kernel_args={"sigma":[sigma],})[0]
kernel01 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[0], reps[1], kernel_args={"sigma":[sigma],})[0]
kernel02 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[0], reps[2], kernel_args={"sigma":[sigma],})[0]
kernel11 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[1], reps[1], kernel_args={"sigma":[sigma],})[0]
kernel12 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[1], reps[2], kernel_args={"sigma":[sigma],})[0]
kernel22 = qml.fchl.fchl_scalar_kernels.get_atomic_kernels(reps[2], reps[2], kernel_args={"sigma":[sigma],})[0]

In [12]:
kernel3x3 = np.block([[kernel00, kernel01, kernel02], [kernel01.T, kernel11, kernel12], [kernel02.T, kernel12.T, kernel22]])

In [13]:
# kernel between molecule at index 0 and molecule at index 2

# select correct indices
start_idx = molecule_size[14] + molecule_size[519]
print(start_idx)
end_idx = start_idx + molecule_size[549] - 1
print(end_idx)

# pick slice
kernel3x3_02 = kernel3x3[0:16, 26:39]

26
38


In [14]:
# load kernel

In [15]:
kernel_full = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig432.8761281083057')

In [16]:
# indices in full kernel
start_idx_full_row = molecule_size[0:14].sum()
end_idx_full_row = start_idx_full_row + molecule_size[14]-1

In [17]:
start_idx_full_column = molecule_size[0:549].sum()
end_idx_full_column= start_idx_full_column + molecule_size[549]-1

In [18]:
# select correct indices
kernel_full_slice = kernel_full[start_idx_full_row:end_idx_full_row+1, start_idx_full_column:end_idx_full_column+1]

In [19]:
kernel02 - kernel_full_slice

ValueError: operands could not be broadcast together with shapes (16,13) (0,0) 

In [ ]:
kernel3x3_02 - kernel_full_slice

## indices are correctly assigned in full matrix
## what about kernel splitting during hyperparameter splitting?

In [ ]:
print(start_idx_full_row)
print(end_idx_full_row)

In [ ]:
# split kernel
tr_kernel, test_kernel = qmi.split_kernel(kernel_full, tr_ind, test_ind)

In [ ]:
# correct kernel selected? yes identical to manually computed kernel
print(np.amax(tr_kernel-kernel3x3))
print(np.amin(tr_kernel-kernel3x3))

# Training error

In [ ]:
import numpy as np
import qml
import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import qml_interface as qmi

In [ ]:
kernel_full1 = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig10000')

In [ ]:
kernel_full2 = np.loadtxt('/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/FCHL/full_kernel_sig432.8761281083057')

In [ ]:
np.amax(kernel_full2-kernel_full1)

In [ ]:
kernel_full1

In [ ]:
tr_kernel, test_kernel = qmi.split_kernel(kernel_full, tr_ind, test_ind)

In [ ]:
# calculate training error
labels = qmi.generate_label_vector(data, molecule_size.sum(), value='atomisation')
tr_label = labels[tr_ind]

coeffs = qml.math.cho_solve(tr_kernel, tr_label)
pred_label = np.dot(tr_kernel, coeffs)
tr_error = np.abs(tr_label-pred_label)

In [ ]:
# training error to high